## Spotify API
Building on the Pitchfork scrape, this notebook loops through the new music entries and adds a Spotify link if one is available,

In [1]:
import pandas as pd
import requests
import time

In [2]:
#Loading in the csv from the last notebook
pitch = pd.read_csv('./new_music.csv')

In [3]:
pitch.head()

,artist,title,review_link,album_track,genre,artwork,preview,score,author
0,The Microphones,Microphones in 2020,https://pitchfork.com//reviews/albums/the-micr...,Album,"['Experimental', 'Rock']",https://media.pitchfork.com/photos/5f284eb719f...,Phil Elverum resurrects his beloved Microphone...,8.5,Quinn Moreland
1,Nazar,“Bunker” [ft. Shannen SP],https://pitchfork.com//reviews/tracks/nazar-bu...,Track,[],https://media.pitchfork.com/photos/5f08b15526e...,For the Manchester-based Angolan electronic ar...,NaN,Eric Torres
2,that dog.,Totally Crushed Out! / Retreat From the Sun,https://pitchfork.com//reviews/albums/that-dog...,Album,['Rock'],https://media.pitchfork.com/photos/5f1efd858fd...,Reissues of the L.A. band’s mid-’90s albums ca...,8.3,Jenn Pelly
3,Dehd,Flower of Devotion,/reviews/albums/dehd-flower-of-devotion/,Album,['Rock'],https://media.pitchfork.com/photos/5f0e1f47763...,"Recording in a studio, the Chicago DIY trio so...",8.3,Steven Arroyo
4,Julianna Barwick,Healing Is a Miracle,/reviews/albums/julianna-barwick-healing-is-a-...,Album,['Electronic'],https://media.pitchfork.com/photos/5f073c85ca5...,The vocalist and producer Juliana Barwick’s re...,8.3,Will Gottsegen


In [4]:
#Combining these fields makes it easier to search
pitch['title_artist'] = pitch['artist'] + ' ' + pitch['title']

In [5]:
#Bringing in Spotify credentials
from credentials import s_key, ss_key

In [6]:
#This function gets an auth token
def auth(key, sec):
    
    authorize = 'https://accounts.spotify.com/api/token'
    param = {
    "Content-Type": "application/x-www-form-urlencoded",
    'grant_type' : 'client_credentials'
    }
    res = requests.post(authorize, auth = (s_key, ss_key), data = param)
    token = res.json()['access_token']
    
    return token

In [7]:
#This function loops through the dataframe, search Spotify, and adds the link to listen if one is available

def get_uri(df):
    #Set up empty counter and total songs to get
    count = 0 
    total = len(df)
    token = auth(s_key, ss_key)
    
    for i in range(len(df)):
        count += 1
        song = df.loc[i, 'title_artist']
       
        try:
            
            #Search parameters with the song plugged in
            params = {
            'q' : song,
            'type': 'album',
            'limit' : 5
            }

            #Header for authorization
            header = {'Authorization' : f'Bearer {token}'}

            #Search endpoint
            spotify_search = 'https://api.spotify.com/v1/search'

            #Make the request
            res = requests.get(spotify_search, headers = header, params = params)
            status = res.status_code
            
            #Continue if status code is 200:
            if res.status_code == 200:
            
                #Save the results and grab the uri
                results = res.json()

                uri = results['albums']['items'][0]['external_urls']['spotify']
            
            else:
                print(df.loc[i, 'title_artist'])
        
        except:
            #If it doesn't work, print out the song and fill in with nan
            print(f'Could not retrieve {song}, status code {status}')
            uri = 'nan'
            
        #Add the info the the DataFrame:
       
        df.loc[i, 'spotify'] = uri
        
        #Print updates for every 100
        if count % 100 == 0:
            print(f'{count} gathered out of {total}')
        
        #Take a break to not exceed the rate limit
        time.sleep(1)

    return df

In [ ]:
pitch = get_uri(pitch)

Could not retrieve The Microphones Microphones in 2020, status code 200
Could not retrieve Nazar “Bunker” [ft. Shannen SP], status code 200
Could not retrieve that dog. Totally Crushed Out! / Retreat From the Sun, status code 200


In [ ]:
#Returnedf dataframe
pitch